# Retrieval-based Customer Support Agent (FAQ + Tickets)

This notebook builds an end-to-end retrieval pipeline for a customer support use case:
- Generate / load a dataset of FAQs + past support tickets
- Clean and chunk text
- Embed text with **Euri** embedding API (example code included; set your API key)
- Store vectors in FAISS (local) or Qdrant (optional example)
- Build a simple retrieval-based chatbot that returns suggested answers

⚠️ **Notes**:
- This notebook contains example code that calls the *Euri* embedding endpoint using `requests`. Replace the placeholder URL and API key with your real values.
- Install dependencies before running the cells (instructions included). When running locally in VS Code, run the notebook with a Python interpreter that has required packages installed.
- The notebook is intended to be runnable as-is after installing dependencies. No remote execution is done by this file itself.


## 1) Requirements / Install

Run these (once) in your environment / terminal before executing the notebook cells:
```bash
pip install pandas numpy scikit-learn faiss-cpu requests qdrant-client sentence-transformers tqdm
```
If you prefer Qdrant instead of FAISS, install and configure Qdrant server and set `USE_QDRANT=True` in the notebook.


## 2) Helper pipeline functions

Below are utility functions used in the pipeline: cleaning, chunking, embedding (Euri example), and FAISS index creation.


In [2]:
import os
import re
import json
import math
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
from tqdm import tqdm

def clean_text(text: str) -> str:
    """Simple cleaning: remove extra whitespace, normalize unicode, strip HTML tags if present."""
    if not isinstance(text, str):
        return ""
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'<[^>]+>', ' ', text)  # remove simple HTML
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def chunk_text(text: str, max_chars: int = 500) -> List[str]:
    """Chunk text into pieces of up to max_chars characters, trying to split on sentence boundaries."""
    text = text.strip()
    if len(text) <= max_chars:
        return [text]
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current = []
    cur_len = 0
    for s in sentences:
        if cur_len + len(s) + 1 <= max_chars:
            current.append(s)
            cur_len += len(s) + 1
        else:
            if current:
                chunks.append(' '.join(current).strip())
            # if sentence itself longer than max_chars, hard-split
            if len(s) > max_chars:
                for i in range(0, len(s), max_chars):
                    chunks.append(s[i:i+max_chars])
                current = []
                cur_len = 0
            else:
                current = [s]
                cur_len = len(s) + 1
    if current:
        chunks.append(' '.join(current).strip())
    return chunks

def flatten_chunks(records: List[Dict], max_chars=500) -> List[Dict]:
    """Take records (each with 'id','text','meta') and produce chunked records with chunk_id and chunk_text."""
    out = []
    for r in records:
        text = clean_text(r.get('text',''))
        chunks = chunk_text(text, max_chars=max_chars)
        for i, c in enumerate(chunks):
            out.append({
                'orig_id': r.get('id'),
                'chunk_id': f"{r.get('id')}_c{i}",
                'text': c,
                'meta': r.get('meta', {})
            })
    return out


## 3) Euri embedding example (replace with your real endpoint & key)

This cell shows how to call Euri's embedding endpoint. Update `EURI_API_KEY` and `EURI_EMBED_URL`.

The function `get_embeddings_euri` accepts a list of strings and returns a numpy array of vectors.


In [ ]:
import requests

EURI_API_KEY = os.environ.get('EURI_API_KEY', '{EURi_API_KEY}')
EURI_EMBED_URL = os.environ.get('EURI_EMBED_URL', 'https://api.euri.example/v1/embeddings')

def get_embeddings_euri(texts: List[str], batch_size: int = 16) -> np.ndarray:
    """Call Euri embedding API and return embeddings as a numpy array.
    **Replace** EURI_EMBED_URL with the actual endpoint and ensure EURI_API_KEY is set in your environment.
    ```
    Example request payload (depends on Euri API; adjust as needed):
    POST /v1/embeddings
    {
      "model": "euri-embed-1",
      "input": ["text1", "text2"]
    }
    ```
    """
    headers = {
        'Authorization': f'Bearer {EURI_API_KEY}',
        'Content-Type': 'application/json',
    }
    embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        payload = {
            'model': 'euri-embed-1',
            'input': batch
        }
        resp = requests.post(EURI_EMBED_URL, headers=headers, json=payload)
        if resp.status_code != 200:
            raise RuntimeError(f'Euri embedding API error {resp.status_code}: {resp.text}')
        data = resp.json()
        # Adjust depending on Euri response format. Common: {"data": [{"embedding": [...]}, ...]}
        for item in data.get('data', []):
            embeddings.append(item['embedding'])
    return np.array(embeddings, dtype=np.float32)


## 4) FAISS index creation & search

The example below creates a FAISS index (IndexFlatIP) using inner product similarity. We normalize vectors to use cosine similarity.


In [6]:
def build_faiss_index(embeddings: np.ndarray):
    try:
        import faiss
    except Exception as e:
        raise RuntimeError('faiss is required. pip install faiss-cpu') from e
    # Normalize for cosine similarity
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    norms[norms==0] = 1e-6
    emb_norm = embeddings / norms
    dim = emb_norm.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(emb_norm.astype(np.float32))
    return index

def search_faiss(index, query_emb: np.ndarray, top_k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
    # normalize
    qn = query_emb / (np.linalg.norm(query_emb) + 1e-10)
    D, I = index.search(qn.reshape(1, -1).astype(np.float32), top_k)
    return D[0], I[0]


## 5) Example dataset generation (FAQs + support tickets)
Creates a small example CSV `support_data.csv` so you can run the pipeline immediately. Replace with your real dataset (CSV/JSON) as needed.


In [16]:
SUPPORT_CSV = './support_data.csv' 

def generate_example_dataset(path=SUPPORT_CSV):
    rows = []
    # FAQs
    rows.append({'id': 'faq_1', 'type': 'faq', 'subject': 'How to reset my password?', 
                 'text': "To reset your password go to Settings -> Account -> Reset password. You'll receive an email with a reset link.", 
                 'answer': "Go to Settings → Account → Reset password. Check your email and follow the link."})
    
    rows.append({'id': 'faq_2', 'type': 'faq', 'subject': 'How to cancel subscription?', 
                 'text': "You can cancel from the Billing page in your account settings or contact support.", 
                 'answer': "Open Billing → Cancel subscription. Contact support if you have trouble."})
    
    rows.append({'id': 'faq_3', 'type': 'faq', 'subject': 'How to update billing information?', 
                 'text': "To update your billing info, go to Account -> Billing -> Update payment method.", 
                 'answer': "Navigate to Billing → Update payment method and enter your new details."})
    
    rows.append({'id': 'faq_4', 'type': 'faq', 'subject': 'Do you offer refunds?', 
                 'text': "Refunds are available within 14 days of purchase if eligible.", 
                 'answer': "Yes, refunds are possible within 14 days. Contact support for help."})
    
    rows.append({'id': 'faq_5', 'type': 'faq', 'subject': 'How to contact support?', 
                 'text': "You can contact support via email at support@example.com or through the Help Center chat.", 
                 'answer': "Email support@example.com or open the Help Center chat."})

    # Past tickets
    rows.append({'id': 'ticket_1001', 'type': 'ticket', 'subject': 'Unable to login after password reset', 
                 'text': "I tried resetting my password but the link expired. I requested a new link twice and still can't login.", 
                 'answer': 'Support provided a new link and advised to clear browser cache.'})
    
    rows.append({'id': 'ticket_1002', 'type': 'ticket', 'subject': 'Billing charged twice', 
                 'text': "I was charged twice for last month. Please refund one charge.", 
                 'answer': 'Refund issued and billing team notified.'})
    
    rows.append({'id': 'ticket_1003', 'type': 'ticket', 'subject': 'Unable to update payment method', 
                 'text': "When I try to update my credit card info, it says invalid card even though it’s valid.", 
                 'answer': 'Asked customer to try another browser and confirmed card was added manually by billing team.'})
    
    rows.append({'id': 'ticket_1004', 'type': 'ticket', 'subject': 'App keeps crashing', 
                 'text': "The mobile app crashes every time I try to open the dashboard.", 
                 'answer': 'Engineering team released a patch. Customer asked to update to the latest version.'})
    
    rows.append({'id': 'ticket_1005', 'type': 'ticket', 'subject': 'Refund request not processed', 
                 'text': "I requested a refund last week but haven’t received confirmation.", 
                 'answer': 'Support confirmed refund was issued and shared transaction ID.'})
    
    rows.append({'id': 'ticket_1006', 'type': 'ticket', 'subject': 'Unable to login after password reset', 
                 'text': "I tried resetting my password but the link expired. I requested a new link twice and still can't login.", 
                 'answer': 'Support provided a new link and advised to clear browser cache.'})
    
    rows.append({'id': 'ticket_1007', 'type': 'ticket', 'subject': 'Billing charged twice', 
                 'text': "I was charged twice for last month. Please refund one charge.", 
                 'answer': 'Refund issued and billing team notified.'})
    
    rows.append({'id': 'ticket_1008', 'type': 'ticket', 'subject': 'Unable to update payment method', 
                 'text': "When I try to update my credit card info, it says invalid card even though it’s valid.", 
                 'answer': 'Asked customer to try another browser and confirmed card was added manually by billing team.'})
    
    rows.append({'id': 'ticket_1009', 'type': 'ticket', 'subject': 'App keeps crashing', 
                 'text': "The mobile app crashes every time I try to open the dashboard.", 
                 'answer': 'Engineering team released a patch. Customer asked to update to the latest version.'})
    
    rows.append({'id': 'ticket_1010', 'type': 'ticket', 'subject': 'Refund request not processed', 
                 'text': "I requested a refund last week but haven’t received confirmation.", 
                 'answer': 'Support confirmed refund was issued and shared transaction ID.'})


    df = pd.DataFrame(rows)
    df.to_csv(path, index=False)
    
generate_example_dataset()
print('Wrote enriched support CSV to', SUPPORT_CSV)


Wrote enriched support CSV to ./support_data.csv


## 6) Full pipeline example: load -> chunk -> embed -> index
Run the cell to perform the pipeline. Ensure `EURI_API_KEY` and `EURI_EMBED_URL` are set (or modify the embedding function to call another provider).


In [18]:
def run_pipeline(csv_path=SUPPORT_CSV, max_chars=500, use_faiss=True):
    df = pd.read_csv(csv_path)
    # build records
    records = []
    for _, r in df.iterrows():
        records.append({'id': r['id'], 'text': r['text'], 'meta': {'type': r['type'], 'subject': r.get('subject',''), 'answer': r.get('answer','')}})
    chunks = flatten_chunks(records, max_chars=max_chars)
    texts = [c['text'] for c in chunks]
    print(f' -> {len(chunks)} chunks to embed')
    emb = get_embeddings_euri(texts)
    print('-> embeddings shape', emb.shape)
    if use_faiss:
        index = build_faiss_index(emb)
        return {'index': index, 'chunks': chunks, 'embeddings': emb}
    else:
        # Example placeholder for Qdrant - not executed here
        return {'index': None, 'chunks': chunks, 'embeddings': emb}

# Note: Do not call run_pipeline() automatically in the notebook file here - call it interactively after setting your API key.


## 7) Retrieval + simple answer suggestion

This function takes a user query, embeds it via Euri, searches FAISS, and returns the top matched chunk texts + original answer (if available).


In [19]:
def retrieve_suggestions(query: str, state: Dict, top_k: int = 5) -> List[Dict]:
    """state must contain 'index', 'chunks' and optionally 'embeddings'"""
    if state.get('index') is None:
        raise RuntimeError('Index missing. Build index first with run_pipeline()')
    q_emb = get_embeddings_euri([query])[0]
    D, I = search_faiss(state['index'], q_emb, top_k=top_k)
    results = []
    for score, idx in zip(D, I):
        c = state['chunks'][int(idx)]
        results.append({'score': float(score), 'chunk_id': c['chunk_id'], 'text': c['text'], 'meta': c['meta']})
    return results

def format_suggestion(results: List[Dict]) -> str:
    out = []
    for r in results:
        meta = r.get('meta', {})
        answer = meta.get('answer')
        out.append(f"- (score={r['score']:.3f}) {r['text']}\n  suggested answer: {answer}")
    return '\n\n'.join(out)


## 8) Retrieval-based chatbot (simple orchestration)
The notebook includes a simple orchestration: for each user question, retrieve top chunks and show suggested answers. You can later pipe these into an LLM prompt for improved finalization.


In [20]:
def chat_loop(state: Dict):
    print('Retrieval-based chatbot. Type "exit" to stop.')
    while True:
        q = input('\nUser question: ')
        if q.strip().lower() in ('exit','quit'):
            break
        results = retrieve_suggestions(q, state, top_k=5)
        print('\nSuggestions:')
        print(format_suggestion(results))
        print('\n---\n')


## 9) Qdrant example (optional)
If you'd rather use Qdrant, start a Qdrant server and use `qdrant-client` to upload vectors. The following is an example snippet (not executed in this notebook automatically):


In [ ]:
# Example Qdrant usage (commented)
qdrant_snippet = '''
from qdrant_client import QdrantClient
client = QdrantClient(url='http://localhost:6333')
collection_name = 'dou_support_faqs'
client.recreate_collection(collection_name, vectors_config={
    'size': embeddings.shape[1],
    'distance': 'Cosine'
})
points = []
for i, emb in enumerate(embeddings):
    points.append({'id': i, 'vector': emb.tolist(), 'payload': chunks[i]['meta']})
client.upsert(collection_name, points)
'''
print(qdrant_snippet)



from qdrant_client import QdrantClient
client = QdrantClient(url='http://localhost:6333')
collection_name = 'support_faqs'
client.recreate_collection(collection_name, vectors_config={
    'size': embeddings.shape[1],
    'distance': 'Cosine'
})
points = []
for i, emb in enumerate(embeddings):
    points.append({'id': i, 'vector': emb.tolist(), 'payload': chunks[i]['meta']})
client.upsert(collection_name, points)



## 10) Where to go from here
- Hook the retrieval results to an LLM (OpenAI, Anthropic, local) to *generate* the final suggested reply using retrieved context.
- Add caching for embeddings and persisted FAISS indexes to avoid re-embedding.
- Add relevance feedback and reranking (BM25 over retrieved docs, or a neural re-ranker).
- Add metadata filters (e.g., only search in `type=ticket` or by product).


---
### Save details
Notebook generated programmatically on 2025-08-21T09:48:38.233718 UTC.
